In [1]:
import requests
import tensorflow as tf
import numpy as np
import json
import base64
import requests
from pprint import PrettyPrinter
import pandas as pd

In [2]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/fakenews-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1721189215'}]}


In [3]:
def prepare_json(text):
    feature_spec = {
        # "title" --> feature key dari data
        "title": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    example = tf.train.Example(
      features=tf.train.Features(feature=feature_spec)).SerializeToString()
    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]
    
    return json.dumps(
        {
            "signature_name":"serving_default",
            "instances": result
        }
    )

In [4]:
def get_predict(text):
    json_data = prepare_json(text)
    response = requests.post("http://localhost:8080/v1/models/fakenews-detection-model:predict", data=json_data)
    predict = response.json()["predictions"][0][0]
    return predict

In [17]:
# text = "GOP Senator Just Smacked Down The Most Punchable Alt-Right Nazi On The Internet" # 1
#text = "May Brexit offer would hurt, cost EU citizens - EU parliament" # 0
text = "covid 19 infection affect people to die"

predict = get_predict(text)

print(f'News Title: {text}')
print(f'REAL' if predict > 0.6 else f'FAKE')
print(f"Result: '{predict}' ")
# Label (0 = fake and 1 = real)

News Title: covid 19 infection affect people to die
FAKE
Result: '0.307915419' 
